In [49]:
import os
import pandas as pd

In [50]:
HOUSING_PATH="E:/gitclone/titanic_dataset"

In [51]:
def load_data(housing_path=HOUSING_PATH):
    train_path,test_path=os.path.join(housing_path,"train.csv"),os.path.join(housing_path,"test.csv")
    return pd.read_csv(train_path),pd.read_csv(test_path)

In [52]:
train_data,test_data=load_data()

In [53]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [55]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [56]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline=Pipeline([
        ("Imputer",SimpleImputer(strategy="median"))
])

In [58]:
class MostFrequentImputer(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        self.most_frequent=pd.Series([X[c].value_counts().index[0] for c in X],
                                     index=X.columns)
        return self
    def transform(self,X):
        return X.fillna(self.most_frequent)

In [59]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline=Pipeline([
    ("imputer",MostFrequentImputer()),
    ("encoder",OneHotEncoder(sparse=False))
])

In [76]:
num_feat=["Age","SibSp","Parch","Fare"]
string_feat=["Pclass","Sex","Embarked"]

from sklearn.compose import ColumnTransformer
preprocess_pipeline=ColumnTransformer([
    ("num pipeline",num_pipeline,num_feat),
    ("cat_pipeline",cat_pipeline,string_feat)
])

In [78]:
X_train=preprocess_pipeline.fit_transform(train_data)
y_train=train_data["Survived"]

In [79]:
#from sklearn.ensemble import RandomForestClassifier
#rf_clf=RandomForestClassifier(n_estimators=100,random_state=42)

In [83]:
'''from sklearn.preprocessing import FunctionTransformer
import numpy as np
def sib_sum(X):
    sib_sum1=X[:,6]+X[:,7]
    return np.c_[X,sib_sum1]

attrib_adder=FunctionTransformer(sib_sum,validate=False)
train_data_mod=attrib_adder.fit_transform(train_data.values)
'''

'from sklearn.preprocessing import FunctionTransformer\nimport numpy as np\ndef sib_sum(X):\n    sib_sum1=X[:,6]+X[:,7]\n    return np.c_[X,sib_sum1]\n\nattrib_adder=FunctionTransformer(sib_sum,validate=False)\ntrain_data_mod=attrib_adder.fit_transform(train_data.values)\n'

In [84]:
'''train_data_mod=pd.DataFrame(
                    train_data_mod,
                    columns=list(train_data.columns)+["sib_sum"]
)
train_data_mod=train_data_mod.drop("SibSp",axis=1)
train_data_mod=train_data_mod.drop("Parch",axis=1)
train_data_mod.head()'''

'train_data_mod=pd.DataFrame(\n                    train_data_mod,\n                    columns=list(train_data.columns)+["sib_sum"]\n)\ntrain_data_mod=train_data_mod.drop("SibSp",axis=1)\ntrain_data_mod=train_data_mod.drop("Parch",axis=1)\ntrain_data_mod.head()'

In [85]:
#X_train1=preprocess_pipeline.fit_transform(train_data_mod)

In [86]:
#y_scores=cross_val_score(rf_clf,X_train1,y_train,cv=3)
#y_scores.mean()